# Import and Preparation

### Import and Structure Data from Excel

In [90]:
# import excel file with room definition and transform information into lists

# import libaries

import pandas as pd 
import re
import openpyxl

# import excel file and specific sheet (in the same folder)

excel_file = "RoomProgramREL.xlsx"

rd = pd.read_excel(excel_file, sheet_name='Function definition')

# convert needed columns into lists

RoomKeys = rd["Name"].tolist()
RoomNumber = rd["Index"].tolist()
RoomType = rd["Type"].tolist()
RoomConnections = rd["Connected"].tolist()
RoomModules = rd["Matrix_Shape"].tolist()
AreaMultiplicator = rd["Area_Person"].tolist()
MinSpace = rd["Min_Space "].tolist()
MaxSpace = rd["Max_Space"].tolist()
Necessity = rd["Necessity"].tolist()


#RoomUnitsPersons = rd["Units_per_Persons"].tolist()


# import parameters from the excel

parameters = pd.read_excel(excel_file, sheet_name='Parameters')["Value"].tolist()

# define parameters for spacial configuration

BModule = parameters[1]
ModuleArea = BModule * BModule

TGrid = parameters[6]

allowed_rooms = [(1, 3), (2, 2), (2, 3), (3, 4), (4, 4), (4, 6)]


# access a specific room

i = 3

print(RoomKeys[i], RoomNumber[i], RoomType[i], RoomConnections[i], RoomModules[i], AreaMultiplicator[i])


Office C_01 1 nan nan 0.2


### Calculating Capacity

In [91]:
# Served People, Required Space

TotalPeopleHealth = 5600
TotalPeopleSport = 3900
TotalPeopleCulture = 4900

# Assumption: People use the Center 8x per Month, daily are 4 Timeslots avaiable: 1 / 30 / 2 = 0.067

ServingFactor = 0.067

# Define in the 3 Areas Health, Culture and Sport the maximum building capacity

ServedPeopleHealth = int(TotalPeopleHealth * ServingFactor)
ServedPeopleSport = int(TotalPeopleSport * ServingFactor)
ServedPeopleCulture = int(TotalPeopleCulture * ServingFactor)
ServedPeopleAverage = int((ServedPeopleHealth + ServedPeopleSport + ServedPeopleCulture)/3)

print(ServedPeopleHealth, ServedPeopleSport, ServedPeopleCulture, ServedPeopleAverage)

TotalNumberRooms = int(len(RoomNumber))
print(TotalNumberRooms)
RoomArea = []
     

# Calculate the needed sqm per functional room dependant on the maximum capacity

for counter in range(0, TotalNumberRooms):

     if "C" in str(RoomNumber[counter]):
          call = AreaMultiplicator[counter] * ServedPeopleCulture
          RoomArea.append(call)
     elif ("T" or "H") in str(RoomNumber[counter]):
          call = AreaMultiplicator[counter] * ServedPeopleHealth
          RoomArea.append(call)
     elif "S" in str(RoomNumber[counter]):
          call = AreaMultiplicator[counter] * ServedPeopleSport
          RoomArea.append(call)
     else:
          call = AreaMultiplicator[counter] * ServedPeopleAverage
          RoomArea.append(call)

# print(RoomArea) 
# print(RoomKeys) 



375 261 328 321
36


# Definitions

In [92]:
# definitions for matrix generation

def scale_matrix(area, module_size, tartan_size):
    x = module_size * module_size + 1.5 * (module_size * tartan_size)
    y = int(area / x)
    return y

# sort the scaling factor

def sort_factor(x, factor):
    
    # sort the list of proportions and get the index of each

    a = sorted(range(len(x)),key=x.__getitem__)
    b = sorted(x)

    # get roommatrix closest to the wanted proportional factor

    res = min(enumerate(b), key=lambda x: abs(factor - x[1]))
    index = res[0]
    index_b = a[index]
    finalscale = x[index_b]
    output = (finalscale, index_b)

    return output

# getmatrix: create a roommatrix from scratch

def get_matrix(a):

    variants = []
    acalc = int(a / 1.8)
    factor = []
    wantedfactor = 1.5

    for i in range(1, (acalc + 1)):
        temp = a / i
        if (temp).is_integer():
            if i > temp:
                variants.append((i, int(temp)))
            else:
                variants.append((int(temp), i))


    for i in range(0, len(variants)):
        factor.append(variants[i][0] / variants[i][1])


    # sort the list of proportions and get the index of each

    a = sort_factor(factor, wantedfactor)
    index = a[1]
    final_mt = variants[index]

    return final_mt

# concider the special cases of the get_matrix function to create more optimized spaces

def apply_matrix(Elements, wantedfactor, tolerance):

    xg = []
    x = []

    if Elements == 0:
        final = ((0, 0))
    elif Elements == 1:
        final = ((1, 1))
    elif Elements == 2:
        final = ((2, 1))  
    elif Elements == 3:
        final = ((3, 1)) 
    else:

        xg.append(get_matrix(Elements))
        x.append(xg[0][0] / xg[0][1])
        final = xg[0]
        helper = 1
        i = x[0]


        while (i > (wantedfactor + tolerance)) or (i < (wantedfactor - tolerance)):

            helper = helper + 1 
            if (Elements - helper) >= 1:
                xg.append(get_matrix((Elements - helper)))
                v = (len(xg)-1)
                x.append(xg[v][0] / xg[v][1])

            xg.append(get_matrix((Elements + helper)))
            v = (len(xg)-1)
            x.append(xg[v][0] / xg[v][1])

            a = sort_factor(x, wantedfactor)
            index = a[1]
            finalscale = a[0]
            final = xg[index]
            i = finalscale
            
    # print(final)
    # print(finalscale)

    return final 

# get a matrix based on the fixed module inputs

def apply_fixed_matrix(Elements, allowed_rooms):

    # calculate usable matrix sizes 

    allowed = []

    for i in range(0, len(allowed_rooms)):
        allowed.append(allowed_rooms[i][0] * allowed_rooms[i][1])
    res_index = min(range(len(allowed)), key=lambda i: abs(allowed[i]- Elements))
    result = allowed_rooms[res_index]

    return result


# check if list contains NaN (unfilled excel cells)

    return final 
def isNaN(num):
    return num != num




# Producing Room Matrix

In [93]:
# produce individual rooms from the information of the excel sheet 


# RoomQuantity provides the number of fixed unites per room type

RoomQuantity = []
RoomModulesXP = []

# the next part assigns the previously gained room sizes a module matrix.If a room module is predefined, it is just duplicated to reach the right amount of surface.
# if there is no room module defined, a matrix is generated based on minimal and maximal room surface. 

for counter in range(0, TotalNumberRooms):    
    RoomModulesXP.append(0)
    RoomQuantity.append(0)
    if isNaN(RoomModules[counter]) and (RoomType[counter] == 1): #check for empty excel cells
          if MinSpace[counter] > RoomArea[counter]: 
              if Necessity[counter] == 1: #if room smaller than minimum but necessary, it will be added with minimum dimensions
                  RoomQuantity[counter] = 1
                  ModuleQuantity = scale_matrix(MinSpace[counter], BModule, TGrid)
                  RoomModulesXP[counter] = apply_fixed_matrix(ModuleQuantity, allowed_rooms)
              else: ##if room smaller than minimum but  not necessary, it will be deleted
                  RoomQuantity[counter] = 0
                  List1 = (0, 0)
                  RoomModulesXP[counter] = List1
          elif RoomArea[counter] > MaxSpace[counter]:#if room is larger than maximum space, it will be divided to produce equal sized rooms close to a relation factor
              factor = int(RoomArea[counter] / MaxSpace[counter])
              RoomQuantity[counter] = factor
              i = RoomArea[counter] / factor
              RoomArea[counter] = i
              ModuleQuantity = scale_matrix(RoomArea[counter], BModule, TGrid)
              RoomModulesXP[counter] = apply_fixed_matrix(ModuleQuantity, allowed_rooms)
          else: #Room will be converted into a matrix close to a relation factor
              ModuleQuantity = scale_matrix(RoomArea[counter], BModule, TGrid)
              RoomModulesXP[counter] = apply_fixed_matrix(ModuleQuantity, allowed_rooms)
              RoomQuantity[counter] = 1
    elif isNaN(RoomModules[counter]) and (RoomType[counter] == 0): # special rooms will be free from the fixed room size spectre
          if MinSpace[counter] > RoomArea[counter]: 
                RoomQuantity[counter] = 1
                ModuleQuantity = scale_matrix(MinSpace[counter], BModule, TGrid)
                RoomModulesXP[counter] = apply_matrix(ModuleQuantity, 1, 0)
          else:
              ModuleQuantity = scale_matrix(RoomArea[counter], BModule, TGrid)
              RoomModulesXP[counter] = apply_matrix(ModuleQuantity, 1, 0)
              RoomQuantity[counter] = 1

# create room if room matrix is already predefined:

for i in range(0, TotalNumberRooms):    
     if not isNaN(RoomModules[i]): #check for empty excel cells
         temp = re.findall(r'\d+', str(RoomModules[i]))
         RoomModulesXP[i] = (int(temp[0])), int(temp[1])
         x = int(RoomModulesXP[i][0])
         y = int(RoomModulesXP[i][1])
         mc = x* y

         if x == 1:
             if y == 1: 
                ModuleQuantity = RoomArea[i] / ModuleArea
             else:
                ModuleQuantity = RoomArea[i] / (mc * ModuleArea + TGrid * BModule * (y- 1))
         elif y == 1:
             if x == 1: 
                ModuleQuantity = RoomArea[i] / ModuleArea
             else:
                ModuleQuantity = RoomArea[i] / (mc * ModuleArea + TGrid * BModule * (x- 1))
         else:
             ModuleQuantity = RoomArea[i] / ((((x- 1)*(TGrid * y * BModule)) + (TGrid * TGrid * ((y- 1)*(x- 1))) + ((y-1)*(TGrid * x * BModule)) + mc) / mc)


         if(ModuleQuantity < mc) and (Necessity[i] == 1):
              RoomQuantity[i] = 1
         else: 
              call = int(ModuleQuantity / mc)
              RoomQuantity[i] = call

        #   print("ModuleQuantity: " + str(ModuleQuantity))
        #   print("Call: " + str(call))
        #   print("Area: " + str(RoomArea[counter]))
        #   print(RoomModulesXP[counter][0])
        #   print(RoomModulesXP[counter][1])
        #   print(x)
        #   print(RoomQuantity[counter])


# check individual room

# i = 8
# print("Room Keys: " + str(RoomKeys[i]))        
# print(RoomArea[i])
# print("Room Modules: " + str(RoomModulesXP[i]))
# print("Rooms Quantity: " + str(RoomQuantity[i]))


print(RoomArea)
print(RoomQuantity)
print(RoomModulesXP)


[38.519999999999996, 22.470000000000002, 9.629999999999999, 32.800000000000004, 1.6400000000000001, 65.60000000000001, 164.0, 229.6, 65.60000000000001, 65.60000000000001, 32.1, 56.175, 28.89, 28.89, 28.89, 65.25, 117.45, 52.2, 56.25, 22.5, 22.5, 56.25, 37.5, 150.0, 37.5, 30.0, 45.0, 18.75, 11.25, 56.25, 3.21, 3.21, 3.21, 3.21, 3.21, 64.2]
[1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3, 1, 1, 1, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 2]
[(3, 3), (2, 3), (1, 3), (3, 4), (1, 3), (5, 5), (8, 8), (10, 10), (4, 6), (4, 6), (3, 4), (4, 6), (2, 4), (2, 4), (2, 4), (4, 6), (4, 6), (4, 6), (5, 5), (2, 2), (2, 2), (4, 6), (4, 4), (8, 8), (4, 4), (1, 4), (4, 4), (3, 4), (2, 3), (4, 6), (1, 1), (1, 3), (1, 1), (1, 1), (1, 1), (4, 4)]


In [94]:
df = rd

df["Rooms_Area"] = RoomArea
df["Room_Count"] = RoomQuantity
df["Rooms_Matrix"] = RoomModulesXP

# df

# Export

In [95]:
# delete sheet on excel

wb = openpyxl.load_workbook(excel_file)
Outputs = wb.get_sheet_by_name('OutputRooms')
wb.remove_sheet(Outputs)
wb.save("RoomProgramREL.xlsx")

# and write data as output to excel
with pd.ExcelWriter("RoomProgramREL.xlsx", mode="a", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="OutputRooms")

C:\CondaTemp/ipykernel_7012/3469007233.py:4: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  Outputs = wb.get_sheet_by_name('OutputRooms')
C:\CondaTemp/ipykernel_7012/3469007233.py:5: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(Outputs)
